# Image Segmentation pipeline

For the purpose of this task we will be using PASCAL VOC datset. The dataset contains a total of 2913 images with segmentation annotations. Code in the cell below will download the code and extract the dataset.

In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar -xvf VOCtrainval_11-May-2012.tar

In [ ]:
!pip install scipy==1.1.0

### 1.1 Loading the dataset

In [ ]:
import os
from os.path import join as pjoin
import collections
import json
import torch
import imageio
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt
import glob

from PIL import Image
from tqdm import tqdm
from torch.utils import data
from torchvision import transforms


class pascalVOCDataset(data.Dataset):
    """Data loader for the Pascal VOC semantic segmentation dataset.

    Annotations from both the original VOC data (which consist of RGB images
    in which colours map to specific classes) and the SBD (Berkely) dataset
    (where annotations are stored as .mat files) are converted into a common
    `label_mask` format.  Under this format, each mask is an (M,N) array of
    integer values from 0 to 21, where 0 represents the background class.

    The label masks are stored in a new folder, called `pre_encoded`, which
    is added as a subdirectory of the `SegmentationClass` folder in the
    original Pascal VOC data layout.

    A total of five data splits are provided for working with the VOC data:
        train: The original VOC 2012 training data - 1464 images
        val: The original VOC 2012 validation data - 1449 images
        trainval: The combination of `train` and `val` - 2913 images
        train_aug: The unique images present in both the train split and
                   training images from SBD: - 8829 images (the unique members
                   of the result of combining lists of length 1464 and 8498)
        train_aug_val: The original VOC 2012 validation data minus the images
                   present in `train_aug` (This is done with the same logic as
                   the validation set used in FCN PAMI paper, but with VOC 2012
                   rather than VOC 2011) - 904 images
    """

    def __init__(
        self,
        root,
        sbd_path=None,
        split="train_aug",
        is_transform=False,
        img_size=512,
        augmentations=None,
        img_norm=True,
        test_mode=False,
    ):
        self.root = root
        self.sbd_path = sbd_path
        self.split = split
        self.is_transform = is_transform
        self.augmentations = augmentations
        self.img_norm = img_norm
        self.test_mode = test_mode
        self.n_classes = 21
        self.mean = np.array([104.00699, 116.66877, 122.67892])
        self.files = collections.defaultdict(list)
        self.img_size = img_size if isinstance(img_size, tuple) else (img_size, img_size)

        if not self.test_mode:
            for split in ["train", "val", "trainval"]:
                path = pjoin(self.root, "ImageSets/Segmentation", split + ".txt")
                file_list = tuple(open(path, "r"))
                file_list = [id_.rstrip() for id_ in file_list]
                self.files[split] = file_list
            self.setup_annotations()

        self.tf = transforms.Compose(
            [
                # add more trasnformations as you see fit
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    def __len__(self):
        return len(self.files[self.split])

    def __getitem__(self, index):
        im_name = self.files[self.split][index]
        im_path = pjoin(self.root, "JPEGImages", im_name + ".jpg")
        lbl_path = pjoin(self.root, "SegmentationClass/pre_encoded", im_name + ".png")
        im = Image.open(im_path)
        lbl = Image.open(lbl_path)
        if self.augmentations is not None:
            im, lbl = self.augmentations(im, lbl)
        if self.is_transform:
            im, lbl = self.transform(im, lbl)
        return im, torch.clamp(lbl, max=20)

    def transform(self, img, lbl):
        if self.img_size == ("same", "same"):
            pass
        else:
            img = img.resize((self.img_size[0], self.img_size[1]))  # uint8 with RGB mode
            lbl = lbl.resize((self.img_size[0], self.img_size[1]))
        img = self.tf(img)
        lbl = torch.from_numpy(np.array(lbl)).long()
        lbl[lbl == 255] = 0
        return img, lbl

    def get_pascal_labels(self):
        """Load the mapping that associates pascal classes with label colors

        Returns:
            np.ndarray with dimensions (21, 3)
        """
        return np.asarray(
            [
                [0, 0, 0],
                [128, 0, 0],
                [0, 128, 0],
                [128, 128, 0],
                [0, 0, 128],
                [128, 0, 128],
                [0, 128, 128],
                [128, 128, 128],
                [64, 0, 0],
                [192, 0, 0],
                [64, 128, 0],
                [192, 128, 0],
                [64, 0, 128],
                [192, 0, 128],
                [64, 128, 128],
                [192, 128, 128],
                [0, 64, 0],
                [128, 64, 0],
                [0, 192, 0],
                [128, 192, 0],
                [0, 64, 128],
            ]
        )

    def encode_segmap(self, mask):
        """Encode segmentation label images as pascal classes

        Args:
            mask (np.ndarray): raw segmentation label image of dimension
              (M, N, 3), in which the Pascal classes are encoded as colours.

        Returns:
            (np.ndarray): class map with dimensions (M,N), where the value at
            a given location is the integer denoting the class index.
        """
        mask = mask.astype(int)
        label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int16)
        for ii, label in enumerate(self.get_pascal_labels()):
            label_mask[np.where(np.all(mask == label, axis=-1))[:2]] = ii
        label_mask = label_mask.astype(int)
        # print(np.unique(label_mask))
        return label_mask

    def decode_segmap(self, label_mask, plot=False):
        """Decode segmentation class labels into a color image

        Args:
            label_mask (np.ndarray): an (M,N) array of integer values denoting
              the class label at each spatial location.
            plot (bool, optional): whether to show the resulting color image
              in a figure.

        Returns:
            (np.ndarray, optional): the resulting decoded color image.
        """
        label_colours = self.get_pascal_labels()
        r = label_mask.copy()
        g = label_mask.copy()
        b = label_mask.copy()
        for ll in range(0, self.n_classes):
            r[label_mask == ll] = label_colours[ll, 0]
            g[label_mask == ll] = label_colours[ll, 1]
            b[label_mask == ll] = label_colours[ll, 2]
        rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
        rgb[:, :, 0] = r / 255.0
        rgb[:, :, 1] = g / 255.0
        rgb[:, :, 2] = b / 255.0
        if plot:
            plt.imshow(rgb)
            plt.show()
        else:
            return rgb

    def setup_annotations(self):
        """Sets up Berkley annotations by adding image indices to the
        `train_aug` split and pre-encode all segmentation labels into the
        common label_mask format (if this has not already been done). This
        function also defines the `train_aug` and `train_aug_val` data splits
        according to the description in the class docstring
        """
        sbd_path = self.sbd_path
        target_path = pjoin(self.root, "SegmentationClass/pre_encoded")
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        train_aug = self.files["train"]

        # keep unique elements (stable)
        train_aug = [train_aug[i] for i in sorted(np.unique(train_aug, return_index=True)[1])]
        self.files["train_aug"] = train_aug
        set_diff = set(self.files["val"]) - set(train_aug)  # remove overlap
        self.files["train_aug_val"] = list(set_diff)

        pre_encoded = glob.glob(pjoin(target_path, "*.png"))
        expected = np.unique(self.files["train_aug"] + self.files["val"]).size

        if len(pre_encoded) != expected:
            print("Pre-encoding segmentation masks...")

            for ii in tqdm(self.files["trainval"]):
                fname = ii + ".png"
                lbl_path = pjoin(self.root, "SegmentationClass", fname)
                lbl = self.encode_segmap(m.imread(lbl_path))
                lbl = m.toimage(lbl, high=lbl.max(), low=lbl.min())
                m.imsave(pjoin(target_path, fname), lbl)

        assert expected == 2913, "unexpected dataset sizes"

### 1.2 Define the model architecture


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Segnet(nn.Module):
  
    def __init__(self):
        super(Segnet, self).__init__()
        #define the layers for your model
        self.conv1 = nn.Conv2d(3, 64, 3, padding = 1)
        self.norm1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 3, padding = 1)
        self.norm2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        self.norm3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)
        self.norm4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)
            
        self.conv5 = nn.Conv2d(256, 512, 3, padding = 1)
        self.norm5 = nn.BatchNorm2d(512)
        
        self.upsample1 = nn.Upsample(scale_factor = 2, mode='bilinear')
        self.conv6 = nn.Conv2d(512, 256, 3, padding = 1)
        self.norm6 = nn.BatchNorm2d(256)
        
        self.upsample2 = nn.Upsample(scale_factor = 2, mode='bilinear')
        self.conv7 = nn.Conv2d(256, 128, 3, padding = 1)
        self.norm7 = nn.BatchNorm2d(128)
        self.conv8 = nn.Conv2d(128, 64, 3, padding = 1)
        self.norm8 = nn.BatchNorm2d(64)
        
        self.conv9 = nn.Conv2d(64, 21, 3, padding = 1)
        
    def forward(self, x):
    #define the forward pass
        x = F.relu(self.conv1(x))
        x = self.norm1(x)
        x = F.relu(self.conv2(x))
        x = self.norm2(x)
        x = self.pool1(x)
        
        x = F.relu(self.conv3(x))
        x = self.norm3(x)
        x = F.relu(self.conv4(x))
        x = self.norm4(x)
        x = self.pool2(x)
        
        
        x = F.relu(self.conv5(x))
        x = self.norm5(x)
        x = self.upsample1(x)
        
        x = F.relu(self.conv6(x))
        x = self.norm6(x)
        x = self.upsample2(x)
        
        x = F.relu(self.conv7(x))
        x = self.norm7(x)
        x = F.relu(self.conv8(x))
        x = self.norm8(x)
        x = F.softmax(self.conv9(x))
        
        return x
    

In [ ]:
# Creating an instance of the model defined above. 
# You can modify it incase you need to pass paratemers to the constructor.
model = Segnet()

### 1.3 Hyperparameters

In [ ]:
local_path = 'VOCdevkit/VOC2012/' # modify it according to your device
bs = 32
epochs = 10

### 1.4 Dataset and Dataloader

In [ ]:
# dataloader variable
trainloader = pascalVOCDataset(local_path)

### 1.5 Loss fuction and Optimizer

In [ ]:
#because of uneven classes
samples = np.array([203289120, 335520, 267840, 227520, 233280, 197280, 132480, 154080, 177120, 259200, 178560, 192960,
          485280, 385920, 1087200, 168334560, 1333440, 216000, 500000, 50000000, 50000000])
samples = 1 - samples/np.sum(samples)
class_weights = torch.FloatTensor(samples)


# loss function
loss_f = nn.CrossEntropyLoss(weight = class_weights)

# optimizer variable
optimizer = torch.optim.SGD(model.parameters(), lr=0.03)


### 1.6 Training the model


In [ ]:
for _ in range(epochs):
    for i, d in enumerate(trainloader):
        if i%32 == 0:
            # new batch 
            images_batch, labels_batch = d
            images_batch = torch.reshape(images_batch, (1,3,512,512))
            labels_batch = torch.reshape(labels_batch, (1,512,512))
        else:
            images_batch = torch.cat((images_batch, torch.reshape(d[0], (1,3,512,512))), 0)
            labels_batch = torch.cat((labels_batch, torch.reshape(d[1], (1,512,512))), 0)
            #after every batch of 32 train
            if (i+1)%32==0:
                optimizer.zero_grad()
                outputs = model(images_batch)
                loss = loss_f(outputs, labels_batch)
                loss.backward()
                optimizer.step()
    #save model
    torch.save(model.state_dict(), "/home/rani00001/checkpoints/epoch"+str(_)+".pt")

### 1.7 Evaluate your model


In [ ]:
def evaluate(ground_truth, predictions):
    
    #flatten arrays to 1D
    ground_truth = torch.flatten(ground_truth).numpy()
    predictions = torch.flatten(predictions).numpy()
    
    if np.unique(ground_truth).shape != np.unique(predictions).shape:
        for i in range(18):
            predictions[i] = i
    predictions[predictions == 18] = 0
    predictions[predictions == 19] = 0
    predictions[predictions == 20] = 0
    
    # calculate metrics
    f1_score = metrics.f1_score(ground_truth, predictions, average = 'micro')
    #make onehot of these arrays for auc score
    onehot_predictions = np.array(pd.get_dummies(predictions))
    onehot_truth = np.array(pd.get_dummies(ground_truth))
    
    auc_score = metrics.roc_auc_score(onehot_truth, onehot_predictions, multi_class = 'ovr')
    dice_coefficient = dice(ground_truth, predictions) 
        
    return f1_score, auc_score, dice_coefficient

def dice(ground_truth, predictions):
    #calculating dice coefficient 
    
    print (np.sum(ground_truth == predictions)/ground_truth.shape[0])
    classes = 21
    for i in range(classes):
        #calculate TP, FP and FN for each class
        TP = np.sum(ground_truth[ground_truth == predictions] == i)
        wrongs = ground_truth != predictions
        FN = np.sum(ground_truth[wrongs] == i)
        FP = np.sum(predictions[wrongs] == i)
        if (TP + FP + FN ==0):
            dice_coefficient = 0
        else:
            dice_coefficient = (2*TP)/((2*TP) + FP + FN)
        
        if i ==0:
            dice_coefficients = np.array(dice_coefficient)
        else:
            dice_coefficients = np.append(dice_coefficients, dice_coefficient)
    return (np.sum(dice_coefficients))/21

### 1.8 Plot the evaluation metrics against epochs


In [ ]:


for epoch in range(epochs):
    model_load = Segnet()
    model_load.load_state_dict(torch.load("/home/rani00001/checkpoints/epoch"+str(epoch)+".pt"))
    with torch.no_grad():
        for i, data in enumerate(trainloader):
            # predict in batches of 32
            if i%32 == 0:
                images_batch, labels_batch = d
                images_batch = torch.reshape(images_batch, (1,3,512,512))
                labels_batch = torch.reshape(labels_batch, (1,512,512))
            else:
                images_batch = torch.cat((images_batch, torch.reshape(d[0], (1,3,512,512))), 0)
                labels_batch = torch.cat((labels_batch, torch.reshape(d[1], (1,512,512))), 0)          
                if (i+1)%32==0:
                    outputs = model_load(images_batch)
                    _, predicted = torch.max(outputs, 1)
                    if i == 31:
                        ground_truth = labels_batch
                        predictions = predicted
                    else:
                        ground_truth = torch.cat((ground_truth, labels_batch), 0)
                        predictions = torch.cat((predictions, predicted), 0)
                        
    f1_score, auc_score, dice_coefficient = evaluate(ground_truth, predictions)
    if epoch == 0:
        f1_scores = np.array(f1_score)
        auc_scores = np.array(auc_score)
        dice_coefficients = np.array(dice_coefficient)

    else:
        f1_scores = np.append(f1_scores, f1_score)
        auc_scores = np.append(auc_scores, auc_score)
        dice_coefficients = np.append(dice_coefficients ,dice_coefficient)       
        
epochs_list = np.arange(epochs)


plt.plot(epochs_list, f1_scores)
strFile = "f1_scores.png"
if os.path.isfile(strFile):
    os.remove(strFile)
plt.savefig(strFile)
plt.show()
plt.close()
plt.plot(epochs_list, auc_scores)
strFile = "auc_scores.png"
if os.path.isfile(strFile):
    os.remove(strFile)
plt.savefig(strFile)
plt.show()
plt.close()
plt.plot(epochs_list, dice_coeficients)
strFile = "dice_coeficients.png"
if os.path.isfile(strFile):
    os.remove(strFile)
plt.savefig(strFile)
plt.show()
plt.close()


### 1.9 Visualize results

In [ ]:
colour_mappings = trainloader.get_pascal_labels()
mean = torch.from_numpy(np.array([0.485, 0.456, 0.406]))
std = torch.from_numpy(np.array([0.229, 0.224, 0.225]))


with torch.no_grad():
        for i, data in enumerate(trainloader):

            if i == 0:
                images_batch, labels_batch = data
                images_batch = torch.reshape(images_batch, (1,3,512,512))
            else:
                images_batch = torch.cat((images_batch, torch.reshape(data[0], (1,3,512,512))), 0)
            if i==9:
                    outputs = model(images_batch)
                    _, predictions = torch.max(outputs, 1)
                    break

segmentation_batch = images_batch

for i in range(segmentation_batch.shape[0]):
    images_batch[i] = torch.add(torch.mul(images_batch[i], std), mean)
    image = transforms.ToPILImage()(images_batch[i])
    
    strFile = "image " + str(i)+".JPEG"
    if os.path.isfile(strFile):
        os.remove(strFile)
    image.save("image " + str(i), "JPEG")
    
    segmentation_batch[i] = images_batch[i]
    
    for j in range(512):
        for k in range(512):
            Class = predictions[i][j][k]
            segmentation_batch[i][:, j, k] = torch.from_numpy(np.asarray(colour_mappings[int(Class)]))
            
    mask = transforms.ToPILImage()(segmentation_batch[i])
    
    strFile = "segmentation mask " + str(i)+".JPEG"
    if os.path.isfile(strFile):
        os.remove(strFile)
    mask.save("segmentation mask " + str(i), "JPEG")